In [1]:
import pandas as pd
import numpy as np

In [2]:
momentum_var, rms_prop_var = None, None

In [3]:
def prepare_df(data):
    data = data.drop(['Name', 'Ticket'], axis=1)
    
    data['Cabin'] = data['Cabin'] == data['Cabin']
    
    for col in ['Sex', 'Embarked', 'Cabin']:
        data = data.join(pd.get_dummies(data[col], prefix=col))
        del data[col]
    
    data = data.dropna(axis=0)
    
    uncols = ['Age', 'Fare', 'Parch', 'SibSp', 'Pclass']
    data[uncols] = (data[uncols] - data[uncols].mean() )/ data[uncols].std()

    return np.array(data.drop(['Survived', 'PassengerId'], axis=1), dtype=np.float16).T, np.array(data['Survived']).reshape(1, -1)

In [4]:
def compute_LogLoss(Yp, Yg):
    cost = -(Yg * np.log(Yp) + (1 - Yg) * np.log(1 - Yp))
    return np.mean(cost)

In [5]:
def forward(Weights, Bias, X, evaluate=False):
    Z = np.dot(Weights.T, X) + Bias
    A = 1. / (1 + np.exp(-(Z)))
    if evaluate == False:
        return A
    if evaluate == True:
        A = A > 0.5
        return A

In [6]:
def backward(pred_Y, grou_Y, Weights, Bias, X, learning_rate ,m_train, beta=0, gd_type='gd'):
    cost = compute_LogLoss(pred_Y.T, grou_Y.T)
    dZ = pred_Y - grou_Y
    dW = np.dot(X, dZ.T) / m_train
    db = np.sum(dZ, axis=1, keepdims=True) / m_train
    
    if gd_type == 'gd':
        Weights -= dW * learning_rate
        Bias -= db * learning_rate

    if gd_type == 'mgd':
        if (momentum_var['vdW'] == 0).all(): momentum_var['vdW'] = dW
        vdW = beta * momentum_var['vdW'] + (1 - beta) * dW
        
        if momentum_var['vdb'] == 0: momentum_var['vdb'] = db
        vdb = beta * momentum_var['vdb'] + (1 - beta) * db
        
        momentum_var['vdW'], momentum_var['vdb'] = vdW, vdb

        Weights -= vdW * learning_rate
        Bias -= vdb * learning_rate
    
    if gd_type == 'rmsprop':        
        if (rms_prop_var['sdW'] == 0).all(): rms_prop_var['sdW'] = abs(dW)
        sdW = beta * rms_prop_var['sdW'] + (1 - beta) * (dW ** 2)
        
        if rms_prop_var['sdb'] == 0: rms_prop_var['sdb'] = db
        sdb = beta * rms_prop_var['sdb'] + (1 - beta) * (db ** 2)
        
        rms_prop_var['sdW'], rms_prop_var['sdb'] = sdW, sdb

        Weights -= learning_rate * dW / np.sqrt(sdW)
        Bias -= learning_rate * db / np.sqrt(sdb)
    
    return cost, Weights, Bias

In [7]:
def accuracy(y_p, y_g):
    y_p = y_p > 0.5
    return np.mean((y_p == y_g))

In [8]:
def train(X, y, val_X, val_y, iters=1000, learning_rate=0.1):
    #     W, b = np.random.rand(X.shape[0], y.shape[0]), 0
    W, b = np.zeros((X.shape[0], y.shape[0])), 0
    m_train = X.shape[1]
    training_records = []
    cost = 0
    for i in range(iters+1):
        pred_y = forward(Weights=W, Bias=b, X=X, evaluate=False)
        cost, W, b = backward(pred_Y=pred_y, grou_Y=y, Weights=W, Bias=b, X=X, learning_rate=learning_rate, m_train=m_train, beta=0.9, gd_type='rmsprop')
        
        if i%(iters//20) == 0: 
            val_pred_y = forward(Weights=W, Bias=b, X=val_X, evaluate=False)
            training_records.append((i, cost, accuracy(pred_y, y), accuracy(val_pred_y, val_y)))
    
    return W, b, training_records
    

In [9]:
data = pd.read_csv('./train.csv')#.sample(frac=1)
train_X, train_Y = prepare_df(data[:700])
valid_X, valid_Y = prepare_df(data[700:])

In [10]:
momentum_var = {
    'vdW' : np.zeros((train_X.shape[0], train_Y.shape[0])),
    'vdb' : 0
}

rms_prop_var = {
    'sdW' : np.zeros((train_X.shape[0], train_Y.shape[0])),
    'sdb' : 0
}

train_X.shape, train_Y.shape, valid_X.shape, valid_Y.shape

((12, 555), (1, 555), (12, 159), (1, 159))

In [11]:
W, b, training_records = train(train_X, train_Y, valid_X, valid_Y, iters=10000, learning_rate=0.01)
training_records_df = pd.DataFrame(training_records, columns=['Iteration', 'Cost', 'Training Accuracy', 'Validation Accuracy'])
training_records_df

,Iteration,Cost,Training Accuracy,Validation Accuracy
0,0,0.693147,0.594595,0.754717
1,500,0.454023,0.796396,0.805031
2,1000,0.454033,0.796396,0.805031
3,1500,0.454013,0.796396,0.805031
4,2000,0.454015,0.798198,0.805031
5,2500,0.454039,0.794595,0.805031
6,3000,0.454029,0.796396,0.805031
7,3500,0.454009,0.796396,0.805031
8,4000,0.453994,0.800000,0.805031
9,4500,0.453984,0.794595,0.805031


In [12]:
training_records_df.to_excel('./results.xlsx')